# Data Querying and Analysis

## Table of Contents
1. Introduction
2. Examples
3. References and Further Reading

<a id='2'></a>
## 1. Introduction

Generative AI can be used in various ways for Natural Language Querying, specifically for converting English to SQL:

1. **Direct Translation**: Converting natural language questions into SQL queries.
2. **Query Optimization**: Improving existing SQL queries based on natural language descriptions.
3. **Schema Understanding**: Interpreting database schemas from natural language descriptions.
4. **Error Correction**: Identifying and fixing errors in SQL queries based on natural language feedback.
5. 1. Query Translation: Converting natural language questions into formal query languages like SQL.
2. Query Generation: Creating complex queries based on high-level descriptions of data needs.
3. Result Interpretation: Translating query results into human-readable explanations.
4. Schema Understanding: Analyzing database schemas to inform query generation.
5. Error Handling: Identifying and explaining errors in queries or results.

Using Gen AI for this task offers several benefits:
- Increased accessibility for non-technical users
- Faster query development
- Reduced errors in complex queries
- Improved data exploration capabilities

**Key Terminology**:
- **Natural Language Processing (NLP)**: The branch of AI that deals with the interaction between computers and humans using natural language.
- **SQL (Structured Query Language)**: A standard language for managing and manipulating relational databases.
- **Schema**: The structure of a database, defining tables, fields, relationships, and constraints.

In [ ]:
!pip install openai pandas scikit-learn matplotlib

In [ ]:
import openai
import os
import json
import pandas as pd
from openai import OpenAI

# Set up OpenAI API key
client = OpenAI(api_key='')

def clean(dict_variable):
    return next(iter(dict_variable.values()))

<a id='3'></a>
## 2. Example 1: Simple Query Conversion

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Convert English queries to SQL."},
        {"role": "user", "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers by revenue."}
    ],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

In [ ]:
schema = """

sales
order_id
customer_id
sales

customer
customer_code
customer_name

"""


response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Convert English queries to SQL."},
        {"role": "user", "content": "Convert this English query to SQL and output in JSON form: Find top 5 customers (customer names) by sales. The schema of this database is: {}".format(schema)}
    ],
    response_format={"type": "json_object"}
)

print(response.choices[0].message.content)

This example demonstrates how Gen AI can convert a simple English query into SQL. The JSON output makes it easy for data engineers to parse and integrate the result into their workflows.

<a id='5'></a>
## 2. Example 2: Query Optimization and Cleaning

In [ ]:
bad_query = """
SELECT * FROM orders WHERE order_date > '2023-01-01' AND customer_id IN (SELECT customer_id FROM customers WHERE country = 'USA')
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Optimize SQL queries based on natural language descriptions."},
        {"role": "user", "content": "Optimize this SQL query and output in JSON form: {} Optimization goal: Improve performance for a large dataset.".format(bad_query)}
    ],
    response_format={"type": "json_object"}
)

print(json.loads(response.choices[0].message.content))

In [ ]:
bad_query = """
SELECT *
FROM (
    SELECT c3.id, c3.date, c3.amt, c3.prod_name, c3.region, e.emp_name
    FROM region c3
    JOIN employees e ON c3.id = e.id
    WHERE e.active = 1
) AS subquery
JOIN (
    SELECT AVG(s.amt) AS avg_amt
    FROM sales s
) AS subquery2 ON subquery.amt > subquery2.avg_amt;
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a SQL expert. Given a SQL query, make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs."},
        {"role": "user", "content": "make it more readable. You may split the query into multiple steps with temp tables, and add naming conventions / column names. \
            Avoid complex joins and nested CTEs. Output in JSON form: {}".format(bad_query)}
    ],
    response_format={"type": "json_object"}
)

good_query = clean(json.loads(response.choices[0].message.content))

In [ ]:
good_query

In [ ]:
for g in good_query:
    print (g)
    print('----')

## 2. Example 3: Query Result Interpretation

In [ ]:
# Simulated query result
query_result = """
| product_category | total_sales |
|-------------------|-------------|
| Electronics      | 1500000     |
| Clothing         | 980000      |
| Home & Garden    | 750000      |
| Books            | 320000      |
| Toys             | 280000      |
"""

prompt = f"Interpret the following query result and provide insights. Here's the result:\n{query_result}"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

<a id='4'></a>
## 2. Example 4: Generating Insights from Query Results

In [ ]:
prompt = """Based on the following query result, suggest 3 relevant follow-up questions for further analysis:
{
  "category": ["Electronics", "Clothing", "Books"],
  "total_sales": [500000, 300000, 100000]
}"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}]
)

print(response.choices[0].message.content)

This example demonstrates how Gen AI can assist in query optimization, a crucial skill for data engineers working with large datasets and complex database structures.

<a id='7'></a>
## 3. References and Further Reading

1. OpenAI API Documentation: https://platform.openai.com/docs/
2. "Natural Language to SQL: A Survey" by Xu et al. (2022): https://arxiv.org/abs/2201.00307
3. "Improving Text-to-SQL Evaluation Methodology" by Zhong et al. (2020): https://arxiv.org/abs/1806.09029
4. SQL Performance Tuning by Peter Gulutzan and Trudy Pelzer
5. "Fundamentals of Database Systems" by Ramez Elmasri and Shamkant Navathe